In [1]:
# Import the necessary Selenium modules.
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

# Set the path to the ChromeDriver executable.
chrome_driver_path = r"D:\C Drive\Program Files\chromedriver-win64\chromedriver.exe"

# Create a Service object for the ChromeDriver using the executable path.
service_obj = Service(chrome_driver_path)

# Initialize a Chrome web driver with the specified service object.
driver = webdriver.Chrome(service=service_obj)

# Import additional modules for later use.
from selenium.webdriver.common.by import By
import time


In [2]:
# LinkedIn Homepage
driver.get("https://www.linkedin.com/home")

In [3]:
mail = "ADD_YOUR_EMAIL_HERE"
password = "ADD_YOUR_PASSWORD_HERE"

In [4]:
# Locate the email input element by its XPath and send keys (enter text) to it.
driver.find_element(By.XPATH, "//input[@type='text']").send_keys(mail)

# Locate the password input element by its XPath and send keys (enter text) to it.
driver.find_element(By.XPATH, "//input[@type='password']").send_keys(password)

# Locate the submit button element by its XPath and click it to submit the form.
driver.find_element(By.XPATH, "//button[@type='submit']").click()


In [12]:
# UK
# driver.get("https://www.linkedin.com/jobs/search/?currentJobId=3724469257&geoId=101165590&location=United%20Kingdom&origin=JOB_SEARCH_PAGE_LOCATION_AUTOCOMPLETE&refresh=true&sortBy=R")

# Denmark
# driver.get("https://www.linkedin.com/jobs/search/?currentJobId=3733759915&geoId=104514075&location=Denmark&origin=JOB_SEARCH_PAGE_LOCATION_AUTOCOMPLETE&refresh=true&sortBy=R")

# Job Posted Page for Scrapping Data
driver.get("https://www.linkedin.com/jobs/search/?currentJobId=3730750033&f_WT=3%2C2&geoId=105117694&location=Sweden&origin=JOB_SEARCH_PAGE_JOB_FILTER&refresh=true&sortBy=DD")


In [13]:
# Import the Pandas library.
import pandas as pd

# Create an empty DataFrame.
df = pd.DataFrame()

# Create empty lists to store data.
job_title = []
name_location = []
job_description = []


In [14]:
# Import necessary modules for Selenium web automation.
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException, StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Initialize page number and consecutive timeout exceptions counters.
page = 1
consecutive_timeout_exceptions = 0

# Begin a loop to navigate through job listing pages.
while True:
    print(page)
    try:
        # Locate and click the page number button (XPath may vary depending on the web page structure).
        # Increase the timeout if required to make sure the element is located
        page_numbers = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, f"//button[@aria-label='Page {page}']"))
        )
        page_numbers.click()
        time.sleep(3)
        
        # Locate job blocks (XPath may vary).
        # Using "find_elements" for searching multiple elements
        jobs_block = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, "//ul[@class='scaffold-layout__list-container']/li"))
        )

        # Scroll job divs into view using JavaScript.
        # executes JS to scroll the job divs into view
        for job in jobs_block:
            try:
                driver.execute_script("arguments[0].scrollIntoView();", job)
                time.sleep(1)
            except StaleElementReferenceException:
                # Handle StaleElementReferenceException by re-locating the element
                job = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, job.get_attribute("xpath"))))
                driver.execute_script("arguments[0].scrollIntoView();", job)
                time.sleep(1)

        # Locate job links (XPath may vary).
        jobs_link = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, "//ul[@class='scaffold-layout__list-container']/li/div/div/div[1]/div[2]/div[1]/a"))
        )

        # Loop through job links.
        for link in jobs_link:
            while True:
                try:
                    link.click()
                    time.sleep(1)

                    # Extract job title, name/location, and job description (XPath may vary).
                    job_title = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, "//h2[@class='t-24 t-bold job-details-jobs-unified-top-card__job-title']"))
                    ).text

                    name_location = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, "//div[@class='job-details-jobs-unified-top-card__primary-description']"))
                    ).text

                    job_description = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, "//div[@id='job-details']"))
                    ).text

                    # Append data to lists.
                    ljob_title.append(job_title)
                    lname_location.append(name_location)
                    ljob_description.append(job_description)
                    break  # Successfully clicked and collected data, so break out of the loop
                except ElementClickInterceptedException:
                    print("Element is not clickable. Retrying...")
                    # Add additional handling here, e.g., scroll to the element
                    driver.execute_script("arguments[0].scrollIntoView();", link)
                except StaleElementReferenceException:
                    print("StaleElementReferenceException. Retrying...")
                    # Re-locate the element and retry the click
                    link = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, link.get_attribute("xpath")))
                                                          )

        # Increment the page number
        page += 1

        # Reset consecutive_timeout_exceptions counter
        consecutive_timeout_exceptions = 0

    except NoSuchElementException as e:
        print(f"Error: {str(e)}")
        traceback.print_exc()
        if "Unable to locate element" in str(e):
            print(f"No more pages after page {page}")
            break
    except TimeoutException as e:
        consecutive_timeout_exceptions += 1
        print(f"TimeoutException: {str(e)} (Consecutive: {consecutive_timeout_exceptions})")
        if consecutive_timeout_exceptions >= 2:
            print("Exceeded consecutive timeout exceptions limit. Exiting.")
            break
        # Handle the TimeoutException here, e.g., refresh the page
        driver.refresh()


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
TimeoutException: Message: 
 (Consecutive: 1)
35
TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF65DE17D32+55474]
	(No symbol) [0x00007FF65DD877E2]
	(No symbol) [0x00007FF65DC3E0EB]
	(No symbol) [0x00007FF65DC7EBAC]
	(No symbol) [0x00007FF65DC7ED2C]
	(No symbol) [0x00007FF65DCB9F77]
	(No symbol) [0x00007FF65DC9F19F]
	(No symbol) [0x00007FF65DCB7EF2]
	(No symbol) [0x00007FF65DC9EF33]
	(No symbol) [0x00007FF65DC73D41]
	(No symbol) [0x00007FF65DC74F84]
	GetHandleVerifier [0x00007FF65E17B772+3609330]
	GetHandleVerifier [0x00007FF65E1D1A90+3962384]
	GetHandleVerifier [0x00007FF65E1C9F1F+3930783]
	GetHandleVerifier [0x00007FF65DEB3CC6+694342]
	(No symbol) [0x00007FF65DD92238]
	(No symbol) [0x00007FF65DD8E4A4]
	(No symbol) [0x00007FF65DD8E5D2]
	(No symbol) [0x00007FF65DD7EE33]
	BaseThreadInitThunk [0x00007FFA1A967344+20]
	RtlUserThreadStart [0x00007FFA1C8426B1+33]
 (Consecutive

In [15]:
# Create a dictionary with data from scraped lists.
data = {
    "job_title": ljob_title,
    "name_location": lname_location,
    "job_description": ljob_description
}

# Create a Pandas DataFrame using the data dictionary.
df = pd.DataFrame(data)

# Print the DataFrame.
print(df)


                                           job_title  \
0                         Group Financial Controller   
1                         Senior 3D Character Artist   
2                                   Graphic Designer   
3    Junior systemingenjör, Medtech - Sectra Medical   
4                            Chief Financial Officer   
..                                               ...   
445            Lead Technical Cloud Engineer (Azure)   
446                   Project Manager Carbon Capture   
447                             Innovationsrådgivare   
448       Information Security Officer, Solna/Hybrid   
449                MEKANIKKONSTRUKTÖR TILL STOCKHOLM   

                                         name_location  \
0    Recap Energy · Stockholm, Stockholm County, Sw...   
1    Gameplay Galaxy · European Economic Area Repos...   
2    Tree Top Staffing LLC · EMEA  56 minutes ago  ...   
3    Sectra · Stockholm, Stockholm County, Sweden R...   
4    Tree Top Staffing LLC · EMEA  57

In [16]:
# Printing the DataFrame
df.head()

,job_title,name_location,job_description
0,Group Financial Controller,"Recap Energy · Stockholm, Stockholm County, Sw...",About the job\nDo you want to work in an excit...
1,Senior 3D Character Artist,Gameplay Galaxy · European Economic Area Repos...,About the job\nDepartment: Art\n\nEmployment T...
2,Graphic Designer,Tree Top Staffing LLC · EMEA 56 minutes ago ...,About the job\nThe ideal candidate will have s...
3,"Junior systemingenjör, Medtech - Sectra Medical","Sectra · Stockholm, Stockholm County, Sweden R...",About the job\nÄr du nyexad ingenjör eller i b...
4,Chief Financial Officer,Tree Top Staffing LLC · EMEA 57 minutes ago ...,About the job\nThe ideal candidate will be res...
...,...,...,...
445,Lead Technical Cloud Engineer (Azure),PRIMA Partners Global · European Union Reposte...,About the job\nLead Technical Cloud Engineer (...
446,Project Manager Carbon Capture,"Fortum · Kumla, Orebro County, Sweden 1 day a...",About the job\nDo you want to revolutionize th...
447,Innovationsrådgivare,Chalmers University of Technology · Gothenburg...,About the job\nVälkommen att söka tjänsten som...
448,"Information Security Officer, Solna/Hybrid","PostNord Sverige · Stockholm, Stockholm County...",About the job\nDo you want to be part of a dyn...


In [17]:
# Specify the file path where you want to save the CSV file.
# Will need to check for language/Translate during EDA
csv_file_path = 'JD_Sweden.csv'  # Provide the desired file path

# Write the DataFrame to a CSV file at the specified file path.
df.to_csv(csv_file_path, index=False)

# Print a message confirming that the data has been written to the CSV file.
print(f"Data has been written to {csv_file_path}")


Data has been written to JD_Sweden.csv
